In [1]:
from yattag import Doc

doc, tag, text = Doc().tagtext()

with tag('h1'):
    text('Hello world!')

print(doc.getvalue())

<h1>Hello world!</h1>


In [2]:
with tag('icecream', id = '2', flavour = 'pistachio'):
    text("This is really delicious.")

In [3]:
print(doc.getvalue())

<h1>Hello world!</h1><icecream id="2" flavour="pistachio">This is really delicious.</icecream>


In [4]:
from yattag import Doc

doc, tag, text = Doc().tagtext()

doc.asis('<!DOCTYPE html>')
with tag('html'):
    with tag('body'):
        text('Hello world!')

print(doc.getvalue())

<!DOCTYPE html><html><body>Hello world!</body></html>


In [16]:
from yattag import Doc
doc, tag, text = Doc().tagtext()
with tag('simple-method', ("result-name", "x")):
    # doc.attr(("result-name", "x"))
    with tag('set'):
        pass

print(doc.getvalue())

<simple-method result-name="x"><set></set></simple-method>


In [39]:
from yattag import Doc
doc, tag, text = Doc().tagtext()
with tag('simple-method', name='testFieldToResult'):
    doc.stag('set', field='resultValue', value='someResultValue')
    doc.stag('set', field='result1', value='dynamicResultName')
    doc.stag('field-to-result', ('field', 'resultValue'), ('result-name', 'constantResultName'))
    doc.stag('field-to-result', ('field', 'resultValue'), ('result-name', '${result1}'))

print(doc.getvalue())

<simple-method name="testFieldToResult"><set field="resultValue" value="someResultValue" /><set field="result1" value="dynamicResultName" /><field-to-result field="resultValue" result-name="constantResultName" /><field-to-result field="resultValue" result-name="${result1}" /></simple-method>


In [41]:
def set_fields(doc, **kwargs):
    for key, value in kwargs.items():
        doc.stag('set', field=key, value=value)
def field_to_result(doc, **kwargs):
    for key, value in kwargs.items():
        doc.stag('field-to-result', ('field', key), ('result-name', value))

def print_doc(doc):
    from yattag import indent

    result = indent(
        doc.getvalue(),
        indentation = '    ',
        newline = '\r\n',
        indent_text = True
    )
    print(result)

doc, tag, text = Doc().tagtext()
with tag('simple-method', name='testFieldToResult'):
    set_fields(doc, resultValue='someResultValue', 
             result1='dynamicResultName')
    field_to_result(doc, resultValue='constantResultName')

print_doc(doc)

<simple-method name="testFieldToResult">
    <set field="resultValue" value="someResultValue" />
    <set field="result1" value="dynamicResultName" />
    <field-to-result field="resultValue" result-name="constantResultName" />
</simple-method>


In [37]:
from yattag import Doc
from yattag import indent
class MinilangProc(object):
    def __init__(self, oc, name):
        self.oc=oc
        self.name=name
        self.doc, self.tag, self.text = Doc().tagtext()
    def set_fields(self, **kwargs):
        for key, value in kwargs.items():
            self.doc.stag('set', field=key, value=value)
    def field_to_result(self, **kwargs):
        for key, value in kwargs.items():
            self.doc.stag('field-to-result', ('field', key), ('result-name', value))
    def desc(self):
        # with self.tag('simple-method', name='testFieldToResult'):            
        result = indent(
            self.doc.getvalue(),
            indentation = '    ',
            newline = '\r\n',
            indent_text = True
        )
        print(result)

In [38]:
ml=MinilangProc(oc, "testFieldToResult")
ml.set_fields(resultValue='someResultValue', 
             result1='dynamicResultName')
ml.field_to_result(resultValue='constantResultName')
ml.desc()

<set field="resultValue" value="someResultValue" />
<set field="result1" value="dynamicResultName" />
<field-to-result field="resultValue" result-name="constantResultName" />


In [25]:
from yattag import indent

result = indent(
    doc.getvalue(),
    indentation = '    ',
    newline = '\r\n',
    indent_text = True
)
print(result)

<simple-method name="testFieldToResult">
    <set field="resultValue" value="someResultValue" />
    <set field="result1" value="dynamicResultName" />
    <field-to-result field="resultValue" result-name="constantResultName" />
    <field-to-result field="resultValue" result-name="${result1}" />
</simple-method>


In [1]:
from py4j.java_gateway import java_import
from sagas.ofbiz.runtime_context import platform

oc = platform.oc
finder = platform.finder
helper = platform.helper

In [28]:
java_import(oc.j, 'com.sagas.generic.*')
director=oc.j.MiniLangDirector(oc.dispatcher)
ctx=director.createServiceMethodContext()
user=ctx.getUserLogin()
print(user)

{'userLoginId': 'system'}


In [29]:
invoker=director.createSimpleMethod(doc.getvalue())
result=invoker.exec(ctx)
print(result)

success


In [31]:
print(invoker.getDefaultSuccessCode())
print(ctx.getResult('constantResultName'))

success
someResultValue


In [42]:
method_xml='''
<simple-method name="testFieldToResult">
    <set field="resultValue" value="someResultValue" />
    <set field="result1" value="dynamicResultName" />
    <field-to-result field="resultValue" result-name="constantResultName" />
    <field-to-result field="resultValue" result-name="${result1}" />
</simple-method>
'''
invoker=director.createSimpleMethod(method_xml)
result=invoker.exec(ctx)
print(result)

success
